In [1]:
#import Libraries
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus


In [2]:
pathForWikiDataset = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
wikiSentences = WikiCorpus(pathForWikiDataset).get_texts()
print(wikiSentences)

c:\python38\lib\site-packages\gensim\utils.py:1268: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)


<generator object WikiCorpus.get_texts at 0x0000023C0E153CF0>


In [3]:
# get Reuters DataSet
nltk.download('punkt')
nltk.download('reuters')
reutersSentences  = reuters.sents()
print(reutersSentences)

[nltk_data] Downloading package punkt to
[nltk_data]     E:\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package reuters to
[nltk_data]     E:\AppData\Roaming\nltk_data...


[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [4]:
def measureProbability(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    total_Word_Count = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord]/=total_Word_Count

In [5]:


def measureSigleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word]/=wordCount



In [6]:


def convertToLower(item):
  if type(item)==str:
    return item.lower()
  return item



In [7]:
sentenceModelLambda1 = defaultdict(lambda: set())
sentenceModelLambda2 = defaultdict(lambda: set())

In [8]:
def measureWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    for word in sentence:
      wordCount+=1
      sentenceModel1[word]+=1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1]+=1
      sentenceModel3[nextWord][previousWord2]+=1
      sentenceModelLambda1[previousWord1].add(nextWord)
      sentenceModelLambda2[previousWord2].add(nextWord)

  return wordCount

In [9]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0))
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))

In [11]:
wikiCount = measureWordCount(sentenceModel1,sentenceModel2,sentenceModel3,wikiSentences)
print(wikiCount)

452944


In [13]:
reutersCount = measureWordCount(sentenceModel1,sentenceModel2,sentenceModel3,reutersSentences)
print(reutersCount)

1720917


In [14]:
measureProbability(sentenceModel2)
measureProbability(sentenceModel3)

In [15]:
total_word = wikiCount + reutersCount
measureSigleWordProbability(sentenceModel1,total_word)

In [16]:
maxProbabilityWords = []
def makeWordSuggestionByTrigram(previousWord2,previousWord1):
  for nextWord in sentenceModelLambda1[previousWord1] & sentenceModelLambda2[previousWord2]:
    naiveBiasTrigramWeight = sentenceModel1[nextWord]*sentenceModel2[nextWord][previousWord1]*sentenceModel3[nextWord][previousWord2]
    maxProbabilityWords.append((nextWord,naiveBiasTrigramWeight))
makeWordSuggestionByTrigram('my','name')
maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
print(*maxProbabilityWords[:10])

('is', 2.684929364088014e-08) ('to', 1.1753701280369164e-09) (',', 5.59438489695973e-10) ('in', 4.6278217257881707e-10) ('and', 4.5063509855288127e-10) ('for', 3.738790783749001e-10) ('would', 3.5383441884561533e-10) ('will', 2.8735045390861837e-10) ('that', 2.0690529880634479e-10) ('or', 1.711572663102755e-10)


In [ ]:
while(True):
    text = input("Enter your line: ")
    if text == "stop":
        print("Ending The Program.....")
        break
    
    else:
        try:
            maxProbabilityWords = []
            text = text.split(" ")
            makeWordSuggestionByTrigram(text[0],text[1])
            maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
            print(*maxProbabilityWords[:10])
            
        except:
            continue